In [2]:
%matplotlib inline

import numpy as np
import xarray as xr
from sklearn.decomposition import PCA
from sklearn.cross_decomposition import CCA
import pandas as pd
import datetime as dt
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import matplotlib.gridspec as gridspec
import geopandas as gpd
plt.rcParams['figure.figsize'] = (15,9)
from rasterstats import zonal_stats
from rasterio.transform import Affine
from scipy.interpolate import griddata
import rasterio
from matplotlib.patches import Rectangle


In [3]:
import cartopy.crs as ccrs

In [4]:
ds1 = xr.open_dataset('hgt.mon.mean.nc')
hgt = ds1.hgt.sel(lat=slice(90,-7.5), lon=slice(182.5, 360),level=slice(6) ,time=slice('1979-01-01','2014-12-31'))
lat_zg = ds1.lat.sel(lat=slice(90, -7.5))
lon_zg = ds1.lon.sel(lon=slice(182.5, 360))

In [102]:
hgt.shape

<xarray.DataArray 'lon' (lon: 71)>
array([182.5, 185. , 187.5, 190. , 192.5, 195. , 197.5, 200. , 202.5, 205. ,
       207.5, 210. , 212.5, 215. , 217.5, 220. , 222.5, 225. , 227.5, 230. ,
       232.5, 235. , 237.5, 240. , 242.5, 245. , 247.5, 250. , 252.5, 255. ,
       257.5, 260. , 262.5, 265. , 267.5, 270. , 272.5, 275. , 277.5, 280. ,
       282.5, 285. , 287.5, 290. , 292.5, 295. , 297.5, 300. , 302.5, 305. ,
       307.5, 310. , 312.5, 315. , 317.5, 320. , 322.5, 325. , 327.5, 330. ,
       332.5, 335. , 337.5, 340. , 342.5, 345. , 347.5, 350. , 352.5, 355. ,
       357.5], dtype=float32)
Coordinates:
  * lon      (lon) float32 182.5 185.0 187.5 190.0 ... 350.0 352.5 355.0 357.5
Attributes:
    units:          degrees_east
    long_name:      Longitude
    actual_range:   [  0.  357.5]
    standard_name:  longitude
    axis:           X

In [103]:
hgt2d = hgt.values
ntime,nlevel, nrow_zg, ncol_zg = hgt2d.shape
print(nrow_zg,ncol_zg)

40 71


In [1]:
a= pd.read_csv("z_anom_t.csv")
b=pd.read_csv("precip_anom_mean.csv")
a= a.drop(['Unnamed: 0'],axis=1)
b=b.drop(['Unnamed: 0'],axis=1)

b['mean']= b.mean(axis=1)
a_t= a.T

NameError: name 'pd' is not defined

In [105]:
a_zg = PCA(n_components=7)
a_zg.fit(a)
z= a_zg.explained_variance_ratio_
sum(z)

0.781259228822743

In [106]:
a2d_pcs =  a_zg.inverse_transform(a_zg.transform(a))

In [107]:
b_pr= PCA(n_components=4)

In [108]:
b_pr.fit(b)
p=b_pr.explained_variance_ratio_
sum(p)

0.9967878348354067

In [109]:
b2d_pcs =  b

In [110]:
cca = CCA(n_components=3)
a_c, b_c = cca.fit_transform(b2d_pcs, a2d_pcs)

In [111]:
cca = CCA(n_components=4)
a_c, b_c = cca.fit_transform(b2d_pcs, a2d_pcs)

In [112]:
r=np.corrcoef(a_c[:, 0], b_c[:, 0])
r1=np.corrcoef(a_c[:, 1], b_c[:, 1])

In [113]:
r

array([[1.        , 0.65475124],
       [0.65475124, 1.        ]])

In [114]:
shapefile= gpd.read_file('greatlakes_subbasins.shp')

In [115]:
b2d_pcs.shape
a2d_pcs.shape

(36, 2840)

In [116]:
A = cca.x_weights_ 
B = cca.y_weights_
BB = np.reshape(B,(nrow_zg,ncol_zg,4),order='F')
BB_1= BB[:,:,0]


In [117]:
cca1= [A[:,0]]

In [118]:
shapefile['cca1']= A[:,0]
shapefile['geometry']

0    POLYGON ((-80.27591 44.09303, -80.26687 44.085...
1    POLYGON ((-79.85601 47.28073, -79.85985 47.239...
2    POLYGON ((-86.11485 46.54744, -86.11092 46.546...
3    POLYGON ((-78.40375 45.35472, -78.40661 45.341...
4    POLYGON ((-88.52978 50.74652, -88.50701 50.692...
Name: geometry, dtype: geometry

In [128]:
fig = plt.figure()
gs = gridspec.GridSpec(2, 2)
gs.update(wspace=0.1, hspace=0.15)

ax2 = fig.add_subplot(gs[0,1],  projection=ccrs.PlateCarree())
x1, y1 = np.meshgrid(lon_zg, lat_zg)
level=np.arange(-0.08,0.1,0.02)
cs = ax2.contourf(x1, y1, BB_1,level,
                        
                        transform=ccrs.PlateCarree(),
                        cmap='RdBu')
    
cb=fig.colorbar(cs, ax=ax2, shrink=0.8, aspect=20)    
ax2.coastlines()
ax2.set_global()
ax2.set_extent([-180,0,-7.5,90],crs=ccrs.PlateCarree())
ax2.set_title('z500 Anomalies CCA1')
ax2.add_patch(rect)
fig.savefig('CCA1_basin')

In [125]:
rect =Rectangle((xmin, ymin), d1, d2, facecolor="none", ec='k', lw=2)

In [126]:
xmax,ymin,xmin,ymax= shapefile.total_bounds
ymax

50.77194690579363

In [127]:
d1=xmax-xmin
d2=ymax-ymin


In [97]:
fig2 = plt.figure()
gs = gridspec.GridSpec(2, 2)
gs.update(wspace=0.1, hspace=0.15)

ax1 = fig2.add_subplot(gs[0,1],  projection=ccrs.PlateCarree())
shapefile.plot(ax=ax1,column ="cca1", cmap = reversed_map,legend=True)
ax1.set_title('Precipitation Anomaly CCA1')
fig2.savefig('cpc_pr')

In [123]:
orig_map=plt.cm.get_cmap('bwr')
reversed_map = orig_map.reversed()

In [131]:
a_c /= a_c.std(axis=0)
b_c /= b_c.std(axis=0)

In [133]:
year = list(range(1979,2015))
fontsize = 25
plt.plot(year, a_c[:,0],"-o", label='Precipitation',linewidth=3)
plt.plot(year, b_c[:,0],"-o", label='Geopotential Height',linewidth=3)
plt.title('CCA 1 Variates: R = ' + str(round(r[0,1],3)),fontdict={'fontsize': fontsize})

plt.vlines(x = [1985,1988,1998,1999,2000,2007,2010,2011], ymin = [-2.5,-2.5,-2.5,-2.5,-2.5,-2.5,-2.5,-2.5], ymax = [3.5,3.5,3.5,3.5,3.5,3.5,3.5,3.5], colors = 'black',
           label = 'La-Nina Years',linewidth=5, alpha=0.25)
plt.legend()
plt.savefig('coupled_pattern Average')


In [126]:
cca_corr= cca.coef_

C:\Users\tameem\AppData\Roaming\Python\Python38\site-packages\sklearn\cross_decomposition\_pls.py:507: FutureWarning: The attribute `coef_` will be transposed in version 1.3 to be consistent with other linear models in scikit-learn. Currently, `coef_` has a shape of (n_features, n_targets) and in the future it will have a shape of (n_targets, n_features).
  warnings.warn(


In [127]:
cca_eigenvalues = np.diag(cca_corr @ cca_corr.T)
cca_variance_ratio = cca_eigenvalues / np.sum(cca_eigenvalues)

In [128]:
cca_variance_ratio

array([0.50763743, 0.10562811, 0.01601042, 0.10937572, 0.26134833])

In [129]:
cca_eigenvalues

array([4.64793756, 0.96713289, 0.14659166, 1.00144607, 2.3929101 ])